In [1]:
!pip install BeautifulSoup4
!pip install lxml 
!pip install geopy

     |████████████████████████████████| 112kB 26.2MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 26.0MB/s eta 0:00:01     |█████████████████▍              | 3.1MB 26.0MB/s eta 0:00:01     |█████████████████████▋          | 3.9MB 26.0MB/s eta 0:00:01     |████████████████████████▎       | 4.4MB 26.0MB/s eta 0:00:01     |█████████████████████████████   | 5.2MB 26.0MB/s eta 0:00:01
     |████████████████████████████████| 112kB 7.1MB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from geopy.geocoders import Nominatim

import folium

In [3]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_Mumbai_Suburban_Railway_stations'
raw_wikipedia_page= requests.get(wikipedia_link).text

#website scraping using beautifulsoup package
soup = BeautifulSoup(raw_wikipedia_page,'lxml')


In [4]:
table=soup.find_all('table')
df = pd.read_html(str(table))
df

[    0                                                  1
 0   †                                   Terminal station
 1   *  Transfer station (excluding transfer to Indian...
 2  †*       Terminal and transfer station to other lines,
        # Station Name                Station Code  \
        #      English        Marathi Station Code   
 0      1       Airoli          ऐरोली          NaN   
 1      2    Ambarnath        अंबरनाथ            A   
 2      3      Ambivli        आंबिवली          NaN   
 3      4      Andheri         अंधेरी          AAD   
 4      5     Asangaon        आसनगांव           AN   
 ..   ...          ...            ...          ...   
 129  131        Sewri          शिवडी          NaN   
 130  132       Taloja  तळोजे पंचानंद          NaN   
 131  133          Pen            पेण          NaN   
 132  134     Rasayani         रसायनी          NaN   
 133  135    Nagothane        नागोठणे          NaN   
 
                                               Line Fast train 

In [108]:
df[1]['Station Name']['English']

geolocator = Nominatim(user_agent="foursquare_agent")

stations_df = pd.DataFrame(columns=['Station name','location'])

for station in df[1]['Station Name']['English']:
    address = station+' station,india'
    location = geolocator.geocode(address)
    if location is not None:
        stations_df = stations_df.append({
            'Station name': station,
            'location': (location.latitude,location.longitude)
        }, ignore_index=True)   

stations_df.head()

,Station name,location
0,Airoli,"(19.158514699999998, 72.99940185331164)"
1,Ambarnath,"(19.1436074, 73.2955345448486)"
2,Ambivli,"(19.267645, 73.1718625)"
3,Andheri,"(19.1196976, 72.8464205)"
4,Asangaon,"(19.4409122, 73.3065183)"


## Venues and categories
We will use Foursquare API to explore venue categories surrounding each station. Venues can be categorized as residential, professional, shopping or leisure. Let's see what venue categories Foursquare identifies.

In [72]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100

In [73]:
categories_url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
# make the GET request
results = requests.get(categories_url).json()

In [64]:
len(results['response']['categories'])

10

we can see there are 10 major category

In [74]:
for category in results['response']['categories']:
    print(category['name'])

Arts & Entertainment
College & University
Event
Food
Nightlife Spot
Outdoors & Recreation
Professional & Other Places
Residence
Shop & Service
Travel & Transport


We will be using the foursquare explore API with catid to query the number of venues of each category in a specific radius.
The response contains a totalResults value for the specified coordinates, radius and category.

In [104]:
def get_venues_count(loc,  categoryId):
    explore_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={}&radius={}&categoryId={}'.format(
                CLIENT_ID,  CLIENT_SECRET, VERSION,",".join(map(str,loc)),1000,categoryId)
    return requests.get(explore_url).json()['response']['totalResults']

In [84]:
#Create new dataframe to store venues data
stations_venues = stations_df.copy()
for category in categories_list:
    stations_venues_df[category[0]] = 0

In [102]:
len(stations_venues_df)

131

In [106]:
for i in range(len(stations_venues_df)):
    print(i,stations_venues.location.iloc[i])
    for c in categories_list:
        stations_venues.loc[i, c[0]] = get_venues_count(stations_venues.location.iloc[i], categoryId=c[1])
stations_venues.to_csv('stations_venues.csv')

0 (19.158514699999998, 72.99940185331164)
1 (19.1436074, 73.2955345448486)
2 (19.267645, 73.1718625)
3 (19.1196976, 72.8464205)
4 (19.4409122, 73.3065183)
5 (19.503751, 73.327362)
6 (19.1669545, 73.2381899)
7 (19.05492845, 72.8405920869825)
8 (19.1438684, 72.9384327)
9 (19.3102682, 72.8530973)
10 (18.9705432, 73.331417)
11 (19.362530800000002, 73.07847467842203)
12 (19.7863383, 72.7925799)
13 (19.229068, 72.8573628)
14 (18.9766219, 72.8327936)
15 (19.01898695, 73.03909472408442)
16 (18.9524563, 72.8174395)
17 (19.0612128, 72.8975909)
18 (18.9398453, 72.83547038112503)
19 (18.9871984, 72.8326716)
20 (19.0517703, 72.8690974)
21 (18.9359567, 72.8273404)
22 (18.9871866, 72.8438965)
23 (18.9949978, 72.8328707)
24 (27.0724152, 75.6453967)
25 (19.9915236, 72.7434083)
26 (19.2494501, 72.8596206)
27 (19.187695, 73.0583421)
28 (19.1887564, 73.0414169)
29 (18.9674767, 72.8445477)
30 (18.8331631, 73.3203444)
31 (19.2180493, 73.0861355)
32 (19.0148811, 72.8279556)
33 (19.1722019, 72.9632085)
34 (19

# Final consolidated data

In [117]:
stations_venues = pd.read_csv('stations_venues.csv', index_col=0)
stations_venues.head()

,Station name,location,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,Airoli,"(19.158514699999998, 72.99940185331164)",0,5,0,13,2,7,15,4,3,3
1,Ambarnath,"(19.1436074, 73.2955345448486)",0,0,0,0,0,0,0,0,0,0
2,Ambivli,"(19.267645, 73.1718625)",0,0,0,1,0,0,2,1,3,4
3,Andheri,"(19.1196976, 72.8464205)",5,8,0,33,7,9,21,4,36,14
4,Asangaon,"(19.4409122, 73.3065183)",0,0,0,6,0,0,0,0,0,1
